In [1]:
pip install transformers


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.1.1 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [3]:
import pandas as pd
import numpy as np
from gensim.models import Word2Vec
from transformers import AutoTokenizer
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report
import joblib

# Load the dataset
file_path = 'sctask_data_dev_v13_utf8.csv'  # Update with your file path
data = pd.read_csv(file_path, encoding='utf-8')

# Encode 'cat_item' as a numerical feature
cat_item_encoder = LabelEncoder()
data['cat_item_encoded'] = cat_item_encoder.fit_transform(data['cat_item'])

# Load Hugging Face tokenizer
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")

# Tokenize the short descriptions
def tokenize_with_hf(text):
    tokens = tokenizer.tokenize(text)
    return [token.lower() for token in tokens]

data['short_description_tokenized'] = data['short_description'].apply(tokenize_with_hf)

# Train a Word2Vec model
w2v_model = Word2Vec(
    sentences=data['short_description_tokenized'], 
    vector_size=100,  # Size of word vectors
    window=5,         # Context window size
    min_count=1,      # Minimum word frequency
    workers=4,        # Number of CPU threads
    sg=1              # Skip-gram model
)

# Generate sentence embeddings by averaging word vectors
def vectorize_sentence(tokens, model, vector_size):
    vectors = [model.wv[word] for word in tokens if word in model.wv]
    if len(vectors) > 0:
        return np.mean(vectors, axis=0)
    else:
        return np.zeros(vector_size)

data['short_description_vector'] = data['short_description_tokenized'].apply(
    lambda tokens: vectorize_sentence(tokens, w2v_model, vector_size=100)
)

# Create the feature matrix
X = pd.DataFrame(data['short_description_vector'].tolist())  # Expand vectors into separate columns
X['cat_item_encoded'] = data['cat_item_encoded']
X.columns = X.columns.astype(str)

# Encode the target 'kbid'
kbid_encoder = LabelEncoder()
data['kbid_encoded'] = kbid_encoder.fit_transform(data['kbid'])
y = data['kbid_encoded']

# Split the data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train a Random Forest Classifier
model = RandomForestClassifier(random_state=42)
model.fit(X_train, y_train)

# Evaluate the model
y_pred = model.predict(X_test)
print(classification_report(y_test, y_pred, target_names=kbid_encoder.classes_))

# Save the encoders, model, and Word2Vec
joblib.dump(model, 'kbid_predictor_model.pkl')
joblib.dump(cat_item_encoder, 'cat_item_encoder.pkl')
joblib.dump(kbid_encoder, 'kbid_encoder.pkl')
w2v_model.save('word2vec_model.bin')

# Usage example
def predict_kbid(cat_item, short_description):
    # Encode the categorical input
    cat_item_encoded = cat_item_encoder.transform([cat_item])[0]
    
    # Tokenize and vectorize the short description
    tokens = tokenizer.tokenize(short_description)
    short_desc_vectorized = vectorize_sentence(tokens, w2v_model, vector_size=100)
    
    # Combine features
    input_features = pd.DataFrame([short_desc_vectorized.tolist()])
    input_features['cat_item_encoded'] = cat_item_encoded
    
    # Align columns with training data
    input_features = input_features.reindex(columns=X.columns, fill_value=0)
    
    # Predict and decode the KBID
    kbid_encoded = model.predict(input_features)[0]
    return kbid_encoder.inverse_transform([kbid_encoded])[0]




                                          precision    recall  f1-score   support

                  Basic_Mainframe_Access       1.00      1.00      1.00        15
                           Clean_Up_RACF       1.00      1.00      1.00        19
                               GuestWifi       1.00      1.00      1.00       109
      Kiteworks_Add_User_Existing_Folder       1.00      1.00      1.00        63
      Kiteworks_Create_New_Secure_Folder       1.00      1.00      1.00        66
          Kiteworks_Remove_Secure_Folder       1.00      1.00      1.00        46
            Off_Boarding_Integrity_Check       1.00      1.00      1.00        15
                                  Others       1.00      1.00      1.00      2654
                 Remove_Mainframe_Access       1.00      1.00      1.00        23
                computer_integrity_check       1.00      1.00      1.00        25
                   corporate_access_d365       1.00      1.00      1.00        52
               

In [ ]:
# Example prediction
example_prediction = predict_kbid("Asset Management", "Dax Studio - Power BI Software Tool Integration")
print("Predicted KBID:", example_prediction)

In [ ]:
To use the saved model for making predictions on new data, you need to:

Load the saved model and encoders.
Load the Word2Vec model.
Prepare the new input data (e.g., tokenize the short_description and encode cat_item).
Align input features with the training data.
Predict and decode the result.
Here’s a step-by-step guide:

Load the Saved Components
First, ensure you’ve saved the following:

Random Forest Model: kbid_predictor_model.pkl
Category Encoder: cat_item_encoder.pkl
KBID Encoder: kbid_encoder.pkl
Word2Vec Model: word2vec_model.bin
Now, load them:

In [ ]:
import joblib
from gensim.models import Word2Vec
from transformers import AutoTokenizer
import pandas as pd
import numpy as np

# Load the saved models and encoders
model = joblib.load('kbid_predictor_model.pkl')
cat_item_encoder = joblib.load('cat_item_encoder.pkl')
kbid_encoder = joblib.load('kbid_encoder.pkl')
w2v_model = Word2Vec.load('word2vec_model.bin')

# Load Hugging Face tokenizer
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")


In [ ]:
# Define Prediction Function
# The prediction function prepares new input data and uses the loaded model to predict.

# python
# Copy code
def vectorize_sentence(tokens, model, vector_size):
    vectors = [model.wv[word] for word in tokens if word in model.wv]
    if len(vectors) > 0:
        return np.mean(vectors, axis=0)
    else:
        return np.zeros(vector_size)

def predict_kbid(cat_item, short_description):
    # Encode the categorical input
    cat_item_encoded = cat_item_encoder.transform([cat_item])[0]
    
    # Tokenize and vectorize the short description
    tokens = tokenizer.tokenize(short_description)
    short_desc_vectorized = vectorize_sentence(tokens, w2v_model, vector_size=100)
    
    # Combine features
    input_features = pd.DataFrame([short_desc_vectorized.tolist()])
    input_features['cat_item_encoded'] = cat_item_encoded
    
    # Align columns with training data
    input_features.columns = input_features.columns.astype(str)  # Ensure all column names are strings
    input_features = input_features.reindex(columns=X_train.columns, fill_value=0)  # Replace X_train with your training data columns
    
    # Predict and decode the KBID
    kbid_encoded = model.predict(input_features)[0]
    return kbid_encoder.inverse_transform([kbid_encoded])[0]

In [ ]:
Example Usage
Here’s how to use the predict_kbid function for a new prediction:

In [ ]:
# Example input
cat_item = "Asset Management"
short_description = "Dax Studio - Power BI Software Tool Integration"

# Predict KBID
predicted_kbid = predict_kbid(cat_item, short_description)
print("Predicted KBID:", predicted_kbid)


In [ ]:
What Happens in the Code
Categorical Encoding:

Encodes the cat_item using the cat_item_encoder.
Text Tokenization:

Tokenizes the short_description using Hugging Face’s tokenizer.
Word Embedding:

Generates the embedding by averaging Word2Vec vectors for the tokens.
Feature Alignment:

Aligns the features with the model's expected structure (e.g., matching column names).
Prediction:

Uses the Random Forest model to predict the encoded kbid.
Decodes the prediction back to the original KBID value.
Ensure Consistency
Verify that cat_item and short_description follow the same preprocessing steps used during training.
Make sure the column alignment (input_features.reindex()) matches the training feature structure.